In [102]:
# Load packages
from util.loggers import createLogHandler
from input_handler.data_preprocessor import data_preprocessor
from input_handler.load_parameters import load_configuration_file, load_parameters
from input_handler.env_setting import run_setting
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import date
from typing import Dict, Optional
import pandas as pd
from pandas.tseries.offsets import MonthEnd

from scipy.stats import norm
from typing import Tuple, Dict


import warnings
warnings.filterwarnings("ignore")


# show all columns and rows of a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [103]:
configPath = Path(r'C:\Users\SA814XM\Engagement\03_KhanBank\kb_ecl_engine\khb_engine\run_config_file.json')
c = load_configuration_file(configPath=configPath)
rc = run_setting(run_config=c)

In [104]:
logger = createLogHandler('ecl_calc', rc.logPath/'Log_file_risk_parameter_generation.log')
logger.info('********** Initiate ECL Engine Calculator **********')

dp = data_preprocessor(context=rc)

# Load parameters
logger.info('Loading all necessary parameters ...')
print('------- Loading parameter files -------') 

try:
        param = load_parameters(parmPath=rc.parmPath)
except Exception as e:
        logger.exception("message")
else:
        logger.info('Parameter loading complete.')

# load scenario data
logger.info('Loading scenario data ...')
print('------- Loading scenario data -------') 

try:
        # npl_data_path = Path("C:\\Users\\SA814XM\\OneDrive - EY\\99_Data_Server\\scenario\\2024Q2\\data")
        npl_data = dp.load_scenario_data(
            data_path=rc.dataPathScen, file_pattern='NPL')
except Exception as e:
        logger.exception("message")
else:
        logger.info('Scenario data loading complete.')


# Load input data
logger.info('Reading ECL input files ...')
print('------- Reading ECL input files -------') 

try:
        instr_df, _, _ = dp.load_input_data(param=param)

        _, _, sa_df, _, _ = dp.run(instr_df=instr_df,
                                   run_scope=rc.run_scope,
                                   param=param)
        
        fs_df, is_error = dp.standardize_data(param_dict=param,
                                      inDataPath=rc.inDataPath,
                                      rawDataName=rc.sa_fs_table_name,
                                      inputDataExt=rc.inputDataExtECL,
                                      dtype_tbl=rc.dtype_tbl,
                                      )

        other_df, is_error = dp.standardize_data(param_dict=param,
                                        inDataPath=rc.inDataPath,
                                        rawDataName=rc.sa_other_debt_table_name,
                                        inputDataExt=rc.inputDataExtECL,
                                        dtype_tbl=rc.dtype_tbl,
                                        )

        repay_df, is_error = dp.standardize_data(param_dict=param,
                                                inDataPath=rc.inDataPath,
                                                rawDataName=rc.repayment_table_name,
                                                inputDataExt=rc.inputDataExtECL,
                                                dtype_tbl=rc.dtype_tbl)
        
except Exception as e:
        logger.exception("message")
else:
        logger.info('Input data read successful.')


------- Loading parameter files -------


------- Loading scenario data -------
------- Reading ECL input files -------


In [105]:
# get param dfs
es_df = param['Economic_sector']
pwa_df = param['pwa']
period = param['CFL_LGD_period'].Period.values[0]

In [106]:
# get class parmas
base_year = rc.prev_yymm
scenario_version = rc.SCENARIO_VERSION

In [107]:
# get in-scope cust list
custs = fs_df['CUST_ID'].unique().tolist()

## Part 1

### Cash flow projection

#### SA params - output to UI

In [108]:
def get_adjustment_factor(df, other_df, cust):
    # Get adjustment factor = bal in sa_table / (bal in other_debt + bal in sa_table)
    other_df_bal = other_df[other_df['CUST_ID'] == cust].TOTAL_DRAWN_BAL_LCY.sum()
    sa_df_bal = df[df['CUST_ID'] == cust].DRAWN_BAL_LCY.sum()
    adj_factor = sa_df_bal / (sa_df_bal + other_df_bal)
    
    return adj_factor


def generate_fy_keys(base_year: int, lookback: int = 2, lookforward: int = 3) -> dict:
    """Generate financial year keys with clear temporal relationships."""
    fy_dict = {}
    
    # Previous years
    for i in range(1, lookback + 1):
        fy_dict[f'prev_{i}'] = date(base_year - i, 12, 31)
    
    # Current year
    fy_dict['curr'] = date(base_year, 12, 31)
    
    # Future years
    for i in range(1, lookforward + 1):
        fy_dict[f'next_{i}'] = date(base_year + i, 12, 31)
    
    return fy_dict


def prepare_financial_data(fs_df, cust):
    """
    Prepare financial data by cleaning ITEM_NAME, ensuring STATEMENT_DATE is in date format,
    and separating the DataFrame into working capital items (wc_df) and cash-related items (cash_df).

    Parameters:
        fs_df (pd.DataFrame): The financial statement DataFrame containing columns 'ITEM_NAME', 'STATEMENT_TYPE', and 'STATEMENT_DATE'.

    Returns:
        wc_df (pd.DataFrame): DataFrame containing working capital items from the balance sheet.
        cash_df (pd.DataFrame): DataFrame containing cash-related items from the income statement.
    """
    # TODO: incorporate the ITEM_NAME check in pre_run_validation
    fs_df_ = fs_df[fs_df['CUST_ID']==cust].copy()
    
    # Clean up ITEM_NAME by stripping spaces and replacing them with underscores
    fs_df_['ITEM_NAME'] = fs_df_['ITEM_NAME'].str.strip().str.replace(' ', '_')
    
    # Ensure STATEMENT_DATE is in date format (not datetime)
    fs_df_['STATEMENT_DATE'] = pd.to_datetime(fs_df_['STATEMENT_DATE']).dt.date
    
    # Get unique ITEM_NAME lists for BS (Balance Sheet) and IS (Income Statement)
    wc_list = fs_df_[fs_df_['STATEMENT_TYPE'] == 'BS']['ITEM_NAME'].unique().tolist()
    cash_list = fs_df_[fs_df_['STATEMENT_TYPE'] == 'IS']['ITEM_NAME'].unique().tolist()
    
    # Pivot the table to reshape it
    fs_df_1 = fs_df_.pivot_table(
        index=['ITEM_NAME'], 
        columns='STATEMENT_DATE', 
        values='ITEM_AMOUNT', 
        aggfunc='sum'
    ).reset_index()
    
    # Separate into working capital DataFrame (wc_df)
    wc_df = fs_df_1[fs_df_1['ITEM_NAME'].isin(wc_list)]
    
    # Separate into cash-related DataFrame (cash_df)
    cash_df = fs_df_1[fs_df_1['ITEM_NAME'].isin(cash_list)]
    
    return wc_df, cash_df


def get_cash_profit(cash_df, fy_dict):
    """Calculate cash-based net profit using rolling window averages and clear temporal references."""
    # Calculate rolling averages using explicit temporal labels
    cash_df[fy_dict['next_1']] = cash_df[[fy_dict['prev_2'], fy_dict['prev_1'], fy_dict['curr']]].mean(axis=1)
    cash_df[fy_dict['next_2']] = cash_df[[fy_dict['prev_1'], fy_dict['curr'], fy_dict['next_1']]].mean(axis=1)
    cash_df[fy_dict['next_3']] = cash_df[[fy_dict['curr'], fy_dict['next_1'], fy_dict['next_2']]].mean(axis=1)
    
    # Initialize results dataframe with clear structure
    cash_profit_df = pd.DataFrame({
        'ITEM_NAME': ['Net_profit', 'Depreciation']
    })
    
    # Unified calculation for each forecast year
    for year_offset in (1, 2, 3):
        year_key = f'next_{year_offset}'
        
        # Explicit column reference for readability
        data = {
            'Sales': cash_df.loc[cash_df['ITEM_NAME'] == 'Sales', fy_dict[year_key]].values[0],
            'COGS': cash_df.loc[cash_df['ITEM_NAME'] == 'COGS', fy_dict[year_key]].values[0],
            'Operating_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Operating_expense', fy_dict[year_key]].values[0],
            'Depreciation_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Depreciation_expense', fy_dict[year_key]].values[0],
            'Other_income': cash_df.loc[cash_df['ITEM_NAME'] == 'Other_income', fy_dict[year_key]].values[0],
            'Other_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Other_expense', fy_dict[year_key]].values[0],
            'Realized_FX_gain_(loss)': cash_df.loc[cash_df['ITEM_NAME'] == 'Realized_FX_gain_(loss)', fy_dict[year_key]].values[0],
            'Interest_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Interest_expense', fy_dict[year_key]].values[0],
            'Taxes': cash_df.loc[cash_df['ITEM_NAME'] == 'Taxes', fy_dict[year_key]].values[0]
        }
        
        # Clear financial calculations
        ebit = (
            data['Sales'] 
            - data['COGS'] 
            - data['Operating_expense'] 
            - data['Depreciation_expense'] 
            + data['Other_income'] 
            - data['Other_expense'] 
            + data['Realized_FX_gain_(loss)']
        )
        
        net_profit = ebit - data['Interest_expense'] - data['Taxes']
        
        # Structured assignment
        cash_profit_df.loc[
            cash_profit_df['ITEM_NAME'] == 'Net_profit', 
            fy_dict[year_key]
        ] = net_profit
        
        cash_profit_df.loc[
            cash_profit_df['ITEM_NAME'] == 'Depreciation', 
            fy_dict[year_key]
        ] = data['Depreciation_expense']
    
    return cash_profit_df


def get_growth_rate(df, numerator_col, denominator_col):
    """
    Helper function of get_changes_in_working_capital
    Calculate growth rate with robust error handling
    """
    result = df[numerator_col] / df[denominator_col] - 1
    return result.replace([np.inf, -np.inf, np.nan], 0)


def get_changes_in_working_capital(wc_df: pd.DataFrame, fy_dict: dict) -> pd.DataFrame:
    """
    Calculate projected changes in working capital using historical growth patterns.
    Applies negative sign convention for asset increases.
    
    Parameters:
        wc_df (pd.DataFrame): Working capital dataframe containing historical values
        fy_dict (dict): Fiscal year mapping with temporal keys ('prev_*', 'curr', 'next_*')
        
    Returns:
        pd.DataFrame: Contains projected working capital changes for future periods
    """
    
    # List of asset items requiring negative sign convention
    ASSET_ITEMS = [
        'Accounts/trade_receivables',
        'Other_receivables',
        'Intercompany_receivables',
        'Inventory',
        'Advance_to_suppliers',
        'Other_short_term_assets'
    ]

    df = wc_df.copy()

    # 1. Calculate historical growth rates
    df['gr_prev_1'] = get_growth_rate(df, fy_dict['prev_1'], fy_dict['prev_2'])
    df['gr_curr'] = get_growth_rate(df, fy_dict['curr'], fy_dict['prev_1'])

    # 2. Project growth rates
    df['gr_next_1'] = df[['gr_prev_1', 'gr_curr']].mean(axis=1)
    df['gr_next_2'] = df[['gr_curr', 'gr_next_1']].mean(axis=1)
    df['gr_next_3'] = df[['gr_next_1', 'gr_next_2']].mean(axis=1)

    # 3. Calculate projected balances
    df['tmp_1'] = df[fy_dict['curr']] * (1 + df['gr_next_1'])
    df['tmp_2'] = df['tmp_1'] * (1 + df['gr_next_2'])
    df['tmp_3'] = df['tmp_2'] * (1 + df['gr_next_3'])

    # 4. Calculate changes
    df[fy_dict['next_1']] = df['tmp_1'] - df[fy_dict['curr']]
    df[fy_dict['next_2']] = df['tmp_2'] - df['tmp_1']
    df[fy_dict['next_3']] = df['tmp_3'] - df['tmp_2']

    # 5. Apply negative sign convention for assets
    asset_mask = df['ITEM_NAME'].isin(ASSET_ITEMS)
    for year in ['next_1', 'next_2', 'next_3']:
        df.loc[asset_mask, fy_dict[year]] *= -1

    return df[['ITEM_NAME', fy_dict['next_1'], fy_dict['next_2'], fy_dict['next_3']]]


def get_sa_params_df(cash_profit_df: pd.DataFrame, ciwc_df: pd.DataFrame, fy_dict: dict) -> pd.DataFrame:
    """
    Calculate strategic analysis parameters for cash flow and working capital changes.
    
    Parameters:
        cash_profit_df (pd.DataFrame): DataFrame containing Net_profit and Depreciation
        ciwc_df (pd.DataFrame): DataFrame with working capital changes
        fy_dict (dict): Fiscal year mapping with 'next_1', 'next_2', 'next_3' keys
        
    Returns:
        pd.DataFrame: Parameters DataFrame with cash and ciwc values for each forecast period
    """
    
    # Validate fiscal year keys
    required_keys = ['next_1', 'next_2', 'next_3']
    if not all(k in fy_dict for k in required_keys):
        missing = [k for k in required_keys if k not in fy_dict]
        raise ValueError(f"Missing fiscal year keys: {missing}")

    # Component definitions
    WC_COMPONENTS = {
        'negative': [
            'Accounts/trade_receivables',
            'Intercompany_receivables',
            'Other_receivables',
            'Inventory',
            'Advance_to_suppliers',
            'Other_short_term_assets'
        ],
        'positive': [
            'Accounts_payables',
            'Intercompany_payables',
            'Tax_payable',
            'Advances_from_customer',
            'Other_current_liabilities'
        ]
    }

    # 1. Calculate cash parameters (Net Profit + Depreciation)
    cash_values = []
    for year_key in required_keys:
        year_col = fy_dict[year_key]
        try:
            np = cash_profit_df.loc[cash_profit_df['ITEM_NAME'] == 'Net_profit', year_col].values[0]
            dp = cash_profit_df.loc[cash_profit_df['ITEM_NAME'] == 'Depreciation', year_col].values[0]
            cash_values.append(np + dp)
        except IndexError:
            raise ValueError(f"Missing Net_profit or Depreciation in {year_key}")

    # 2. Calculate working capital changes
    ciwc_values = []
    for year_key in required_keys:
        year_col = fy_dict[year_key]
        try:
            negative = ciwc_df.loc[ciwc_df['ITEM_NAME'].isin(WC_COMPONENTS['negative']), year_col].sum()
            positive = ciwc_df.loc[ciwc_df['ITEM_NAME'].isin(WC_COMPONENTS['positive']), year_col].sum()
            ciwc_values.append(positive + negative)
        except KeyError:
            raise ValueError(f"Missing working capital components in {year_col}")

    # 3. Construct final DataFrame
    return pd.DataFrame({
        'Parameter': ['cash', 'ciwc'],
        fy_dict['next_1']: [cash_values[0], ciwc_values[0]],
        fy_dict['next_2']: [cash_values[1], ciwc_values[1]],
        fy_dict['next_3']: [cash_values[2], ciwc_values[2]]
    })
    


In [109]:
######## SA CFL LGD Params generation ########

fy_dict = generate_fy_keys(int(str(rc.prev_yymm)[:4]))

sa_params_list = []

# TODO the starting point of the for loop: for cust in custs:
for cust in custs:
    
    sa_df_cust = sa_df[sa_df['CUST_ID'] == cust].copy()
    
    adj_factor = get_adjustment_factor(sa_df_cust, other_df, cust)
    
    wc_df, cash_df = prepare_financial_data(fs_df, cust)

    cash_profit_df = get_cash_profit(cash_df, fy_dict)
    
    ciwc_df = get_changes_in_working_capital(wc_df, fy_dict)
    
    cust_params_df = get_sa_params_df(cash_profit_df, ciwc_df, fy_dict)

    # Insert the "Cust_id" column as the first column and fill it with "123"
    cust_params_df.insert(0, "Cust_ID", cust)
    
    sa_params_list.append(cust_params_df)
    
sa_params_df = pd.concat(sa_params_list, ignore_index=True)
sa_params_df = sa_params_df.reset_index(drop=True)

sa_params_df


,Cust_ID,Parameter,2024-12-31,2025-12-31,2026-12-31
0,DNINULZAV,cash,3.291386e+10,4.423381e+10,4.302075e+10
1,DNINULZAV,ciwc,-3.097999e+10,-2.992405e+11,-1.016467e+12
2,DNWGQVRAQ,cash,2.558667e+08,3.444889e+08,4.410185e+08
3,DNWGQVRAQ,ciwc,-1.015363e+09,-1.598201e+09,-2.053788e+09


## Part 2

### Cash flow projection (cont.)

#### Adjusted SA params -> intput -> get net operating cashflow

In [118]:
def get_net_operating_cashflow(sa_params_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates all parameter rows into a 'noc' (Net Operating Cash) row
    
    Parameters:
        sa_params_df: DataFrame with columns ['Parameter', 'next_1', 'next_2', 'next_3']
        
    Returns:
        DataFrame with original rows plus new 'noc' row containing column sums
        
    Example Input:
        | Parameter | next_1 | next_2 | next_3 |
        |-----------|--------|--------|--------|
        | cash      | 150    | 165    | 182    |
        | ciwc      | -50    | -55    | -60    |
        
    Example Output:
        | Parameter | next_1 | next_2 | next_3 |
        |-----------|--------|--------|--------|

        | noc       | 100    | 110    | 122    |
    """
    # Create a copy to avoid modifying original data
    df = sa_params_df.copy()
    
    # Calculate sums for each forecast period
    noc_values = {
        'Parameter': 'noc',
        fy_dict['next_1']: df[fy_dict['next_1']].sum(),
        fy_dict['next_2']: df[fy_dict['next_2']].sum(),
        fy_dict['next_3']: df[fy_dict['next_3']].sum()
    }
    
    # Append new row using concat for pandas>=1.4.0 compatibility
    noc_row = pd.DataFrame([noc_values])
    return noc_row


def _calculate_npl_stats(data: pd.Series, alpha: float) -> Tuple[float, float, float]:
    """Helper function to calculate NPL statistics"""
    n = len(data)
    mean = data.mean()
    std = data.std()
    
    z_score = norm.ppf(1 - alpha/2)
    std_err = std / (n ** 0.5)

    return (
        mean,
        mean - z_score * std_err,
        mean + z_score * std_err
    )


def calculate_sector_npl_stats(
    es_df: pd.DataFrame,
    npl_data: pd.DataFrame,
    alpha: float = 0.1
) -> pd.DataFrame:
    """
    Calculate NPL statistics with confidence intervals for economic sectors.
    
    Args:
        es_df: DataFrame containing economic sector information with 'ECON_SECTOR_TYPE' column
        npl_data: DataFrame containing NPL data with columns named 'SECTOR_NPL_{sector}'
        alpha: Significance level for confidence intervals (default: 0.1)
    
    Returns:
        DataFrame with columns ['sector', 'mean', 'ci_lower', 'ci_upper']
    """
    # Validate inputs
    if 'ECON_SECTOR_TYPE' not in es_df:
        raise ValueError("es_df must contain 'ECON_SECTOR_TYPE' column")
    
    if not 0 < alpha < 1:
        raise ValueError("Alpha must be between 0 and 1")

    # Generate sector names
    sectors = es_df['ECON_SECTOR_TYPE'].unique().tolist()
    npl_columns = [f'SECTOR_NPL_{sec}' for sec in sectors]
    
    # Verify all required columns exist in npl_data
    missing_cols = [col for col in npl_columns if col not in npl_data]
    if missing_cols:
        raise ValueError(f"Missing NPL columns in npl_data: {missing_cols}")

    # Calculate statistics for each sector
    stats = []
    for sector, col in zip(sectors, npl_columns):
        data = npl_data[col].dropna()
        if len(data) < 2:
            raise ValueError(f"Insufficient data for sector {sector}")
            
        mean, ci_lower, ci_upper = _calculate_npl_stats(data, alpha)
        stats.append({
            'npl_sector': sector,
            'mean': mean,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        })

    return pd.DataFrame(stats)


def get_total_repay_df(df, repay_df):
    contract_ids = df.CONTRACT_ID.unique().tolist()
    cust_repay = repay_df[repay_df['CONTRACT_ID'].isin(contract_ids)]
    
    ttl_repay = cust_repay.pivot_table(index='CF_DATE', columns='CONTRACT_ID', values='PRIN_PMT_OCY')
    ttl_repay['ttl_repay'] = ttl_repay.sum(axis=1)
    
    return ttl_repay[['ttl_repay']]


def get_cash_flow_dates(scenario_date: str, period_years: int) -> pd.DatetimeIndex:
    """
    Generate monthly cash flow dates starting from the quarter following the scenario date.
    
    Args:
        scenario_date: Format 'YYYYQX' where X is quarter (1-4)
        period_years: Number of years to project forward
        
    Returns:
        Monthly DatetimeIndex through end of projection period
        
    Example:
        >>> get_cash_flow_dates('2023Q4', 5)
        DatetimeIndex(['2024-01-31', '2024-02-29', ..., '2027-12-31'], dtype='datetime64[ns]', freq='M')
    """
    # Validate input format
    if len(scenario_date) != 6 or not scenario_date[4] == 'Q':
        raise ValueError("Scenario date must be in format 'YYYYQX' (e.g. '2023Q4')")
    
    year = int(scenario_date[:4])
    quarter = int(scenario_date[-1])
    
    if not 1 <= quarter <= 4:
        raise ValueError("Quarter must be between 1-4")

    # Calculate start date
    if quarter == 4:
        start_year = year + 1
        start_month = 1
    else:
        start_year = year
        start_month = quarter * 3 + 1  # Next quarter start month
        
    start_date = pd.Timestamp(year=start_year, month=start_month, day=1) + MonthEnd(1)
    
    # Calculate end date
    end_date = pd.Timestamp(year + period_years - 1, 12, 31)
    
    return pd.date_range(start=start_date, end=end_date, freq='M')


def get_pwa_noc(
    fy_dict: Dict[str, int],
    sa_df_cust: pd.DataFrame,
    noc_df: pd.DataFrame,
    npl_stats: pd.DataFrame,
    pwa_df: pd.DataFrame,
    cust: str
) -> Optional[pd.DataFrame]:
    """
    Calculate Probability-Weighted Average Net Operating Cash (NOC) for a customer.
    
    Args:
        fy_dict: Fiscal year mapping {'next_1': 2024, ...}
        sa_df_cust: Strategic analysis dataframe with customer-sector mapping
        noc_df: NOC projections dataframe
        npl_stats: Sector NPL statistics with confidence intervals
        pwa_df: Scenario probability weights
        cust_id: Target customer ID
        
    Returns:
        DataFrame with PWA NOC projections or None if invalid input
    """
        
    sectors = sa_df_cust['ECON_SECTOR'].unique()
    if len(sectors) > 1:
        print(f"Customer {cust} has multiple sectors: {sectors}")
        return None
    
    # print(sectors[0])
    sector_type = es_df.loc[es_df['ECON_SECTOR'] == sectors[0], 'ECON_SECTOR_TYPE'].iloc[0]
    
    # Get NPL risk factor
    try:
        sector_stats = npl_stats.query(f"npl_sector == '{sector_type}'").iloc[0]
        risk_factor = (sector_stats['mean'] - sector_stats['ci_lower']) / sector_stats['mean']

    except IndexError:
        print(f"No NPL stats found for sector {sector_type}")
        return None

    # Calculate scenario-adjusted NOC
    results = []
    for year_key in ['next_1', 'next_2', 'next_3']:
        if year_key not in fy_dict:
            print(f"Missing fiscal year key {year_key}")
            return None
            
        fiscal_year = fy_dict[year_key]
        base_noc = noc_df.get(fiscal_year, 0)
        
        # Scenario adjustments
        scenarios = {
            'SEVE': base_noc * (1 - risk_factor),
            'BASE': base_noc,
            'GROW': base_noc * (1 + risk_factor)
        }
        
        # Apply probability weights
        pwa_noc = sum(
            scenarios[scenario] * pwa_df.loc[pwa_df['scenario'] == scenario, 'pwa'].values[0]
            for scenario in ['SEVE', 'BASE', 'GROW']
        )

        results.append({'prediction_year': fiscal_year, 'pwa_noc': pwa_noc.iloc[0]
})
    
    return pd.DataFrame(results)


def get_total_exposure(df):
    return df['PRIN_BAL_LCY'].sum() + df['ACRU_INT_LCY'].sum()


def get_cust_EIR(df):
    eir = df['EFF_INT_RT']
    exposure = df['PRIN_BAL_LCY'] + df['ACRU_INT_LCY']

    return (eir * exposure).sum() / exposure.sum()


def get_lgd_df(cash_flow_dates: pd.DatetimeIndex, pwa_noc_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create Loss Given Date (LGD) DataFrame with datetime index preservation.
    
    Args:
        cash_flow_dates: Monthly cash flow dates (DatetimeIndex)
        pwa_noc_df: Annual NOC projections with prediction years
        
    Returns:
        DataFrame with cash flow dates as index and total NOC values
    """
    # Convert prediction years to datetime and extract year
    annual_noc = pwa_noc_df.copy()
    annual_noc['year'] = pd.to_datetime(annual_noc['prediction_year']).dt.year

    # Create base dataframe with cash flow dates as index
    lgd_df = pd.DataFrame(index=cash_flow_dates)
    lgd_df.index.name = 'cash_flow_date'
    
    # Extract year from index and prepare for merge
    lgd_df = lgd_df.assign(year=lgd_df.index.year).reset_index()
    
    # Merge with annual NOC values
    lgd_df = lgd_df.merge(
        annual_noc[['year', 'pwa_noc']],
        on='year',
        how='left'
    )
    
    # Restore datetime index and clean up
    lgd_df = lgd_df.set_index('cash_flow_date').sort_index()
    lgd_df = lgd_df[['pwa_noc']].rename(columns={'pwa_noc': 'ttl_noc'})
    
    # Forward fill within each year group and fill remaining NAs
    lgd_df['ttl_noc'] = lgd_df['ttl_noc'].ffill().fillna(0)
    
    return lgd_df


def add_discounted_col(
    df: pd.DataFrame,
    lgd_df: pd.DataFrame,
    col: str,
    eir: float
) -> pd.DataFrame:
    """
    Add discounted NOC column using time-aware discounting.
    
    Args:
        lgd_df: DataFrame with cash_flow_date index and ttl_noc column
        reporting_date: Base date for discounting (e.g. '2024-06-30')
        eir: Effective interest rate (e.g. 0.05 for 5%)
        
    Returns:
        Modified DataFrame with discounted_noc column
    """
    # Convert reporting date to pandas timestamp if not already
    # TODO: chk if the report_date of one cust is unique and = the scenario date
    reporting_date = df['REPORT_DATE'].unique()[0]
    
    # Calculate time delta in years
    delta_days = (lgd_df.index - reporting_date).days
    delta_years = delta_days / 365
    
    # Calculate discount factor
    discount_factor = (1 + eir) ** delta_years
    
    # Calculate discounted NOC
    lgd_df[f'discounted_{col}'] = lgd_df[col] / discount_factor
    
    return lgd_df


def get_total_recovery(lgd_df_1):
    """
    Calculate the total recoverable amount (delayed repayment plan version)
    New logic: If the cash flow is insufficient in a given month, the entire repayment plan is shifted back by one month until the cumulative cash flow is sufficient to cover the repayment.
    """
    total_recovery = 0.0
    carry_over = 0.0      # Cumulative available cash flow
    pending_repayments = []  # Queue for pending repayment plans
    
    # Process data in chronological order
    df = lgd_df_1.sort_index()
    
    for date, row in df.iterrows():
        discounted_noc = row["discounted_ttl_noc"]
        ttl_repay = row["discounted_ttl_repay"]
        
        # Add the current month's repayment plan to the queue
        pending_repayments.append(ttl_repay)
        
        # Handle negative cash flow
        current_noc = max(discounted_noc, 0.0)
        carry_over += current_noc  # Accumulate cash flow
        
        # Attempt to process the first repayment plan in the queue
        while len(pending_repayments) > 0 and carry_over >= pending_repayments[0]:
            # Repay the earliest outstanding debt
            repaid = pending_repayments.pop(0)
            total_recovery += repaid
            carry_over -= repaid
    
    return total_recovery


In [120]:
# TODO: update the adj_sa_params_df
sa_params_df_adj = sa_params_df.copy()

for cust in custs:
    sa_df_cust = sa_df[sa_df['CUST_ID'] == cust].copy()
    
    ### Adjusted SA params -> intput -> get net operating cashflow
    cust_df = sa_params_df_adj[sa_params_df_adj['Cust_ID'] == cust].copy()
    noc_df = get_net_operating_cashflow(sa_params_df=cust_df)
    
    ### LGD calculation
    
    # NPL statistics
    npl_stats = calculate_sector_npl_stats(es_df, npl_data, alpha=0.1)

    # LGD
    ttl_repay_df = get_total_repay_df(sa_df_cust, repay_df)
    cash_flow_dates = get_cash_flow_dates(rc.SCENARIO_VERSION, period)
    
    # TODO: change the noc_df back after the params are decided
    noc_df_fake = pd.DataFrame({
        'Parameter': ['noc'],
        fy_dict['next_1']: [150033428750.2],
        fy_dict['next_2']: [13410669664.88],
        fy_dict['next_3']: [67990473450.6879]
    })

    
    pwa_noc_df = get_pwa_noc(fy_dict, sa_df_cust, noc_df, npl_stats, pwa_df, cust)
    
    ttl_exposure = get_total_exposure(sa_df_cust)
    
    eir = get_cust_EIR(sa_df_cust)

    lgd_df = get_lgd_df(cash_flow_dates, pwa_noc_df)

    lgd_df_1 = lgd_df.merge(ttl_repay_df, how='left', left_index=True, right_index=True).copy()
    lgd_df_1 = lgd_df_1.fillna(0)  # Fill NaN values with 0

    
    lgd_df_2 = add_discounted_col(
        sa_df_cust,
        lgd_df_1,
        'ttl_noc',
        eir=eir
    )

    lgd_df_2 = add_discounted_col(
        sa_df_cust,
        lgd_df_1,
        'ttl_repay',
        eir=eir
    )
    
    ttl_recover = get_total_recovery(lgd_df_1)
    lgd = 1 - ttl_recover/ttl_exposure

    print(cust, lgd)

DNINULZAV 0.5421736916332003
DNWGQVRAQ 1.0
